In [6]:
import requests
import json
import getpass


## Connect to the musixmatch API to get lyrics:
You can sign up for a free developer account and generate a personal API key

https://developer.musixmatch.com/

In [7]:
api_key = getpass.getpass("api_key")

Defining some helper functions to collect lyrics

In [8]:
def get_songs(artist):
    api_url = "http://api.musixmatch.com/ws/1.1/track.search&apikey=" + api_key 
    url = api_url + "&q_artist=" + str(artist)
    x = requests.get(url)
    response = json.loads(x.text)

    return response['message']["body"]["track_list"]

In [9]:
# len("\n...\n\n******* This Lyrics is NOT for Commercial use *******") = 59

def get_lyrics(track_id):
    api_url = "http://api.musixmatch.com/ws/1.1/track.lyrics.get?apikey=" + api_key
    url = api_url + "&track_id="  + str(track_id)
    x = requests.get(url)
    response = json.loads(x.text)
    lyrics = response['message']["body"]["lyrics"]["lyrics_body"][:-59].split("\n")
    inner_hits = []
    for line in lyrics:
        if line not in ["", '...']:
            inner_hits.append({"line" : line})
    return inner_hits

In [10]:
songs = get_songs("hozier")

song_list = []
for song in songs:
    mysong = {}
    mysong["id"] = song["track"]["track_id"]
    mysong["name"] = song["track"]["track_name"].split("(")[0].split("-")[0]
    mysong["artist"] = song["track"]["artist_name"]
    if (song["track"]["has_lyrics"] == 1):
        mysong["lyrics"] = get_lyrics(mysong["id"])
    else:
        mysong["lyrics"] = None
    song_list.append(mysong)

In [11]:
with open('data/hozier_songs.json', 'w') as f:
    json.dump(song_list, f)

## Pydantic Model example for validating data input

In [12]:
import json
with open('data/hozier_songs.json', 'r') as f:
  songs = json.load(f)

In [13]:
song = songs[0]
song

{'id': 84358250,
 'name': 'Someone New ',
 'artist': 'Hozier',
 'lyrics': [{'line': 'Go take this the wrong way'},
  {'line': 'You knew who I was with every step that I ran to you'},
  {'line': 'Only blue or black days'},
  {'line': 'Electing strange perfections in any stranger I choose'},
  {'line': 'Would things be easier if there was a right way'},
  {'line': 'Honey there is no right way'},
  {'line': "And so I fall in love just a little ol' little bit"},
  {'line': 'Every day with someone new'},
  {'line': "I fall in love just a little ol' little bit"},
  {'line': 'Every day with someone new'},
  {'line': "I fall in love just a little ol' little bit"},
  {'line': 'Every day with someone new'},
  {'line': "I fall in love just a little ol' little bit"},
  {'line': 'Every day with someone new'},
  {'line': "There's an art to life's distractions"},
  {'line': 'To somehow escape the burning weight'},
  {'line': 'The art of scraping through'},
  {'line': 'Some like to imagine'},
  {'line

In [15]:
from pydantic import BaseModel

class Song(BaseModel):
    name: str
    artist: str
    lyrics: list | None
    id : int

    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "name": "Take me to church",
                    "artist": "Hozier",
                    "lyrics": "Her eyes and words are so icy\nOh but she burns\nLike rum on the fire",
                    "id": 0,
                }
            ]
        }
    }
    
example = Song.model_validate(song)

In [16]:
example

Song(name='Someone New ', artist='Hozier', lyrics=[{'line': 'Go take this the wrong way'}, {'line': 'You knew who I was with every step that I ran to you'}, {'line': 'Only blue or black days'}, {'line': 'Electing strange perfections in any stranger I choose'}, {'line': 'Would things be easier if there was a right way'}, {'line': 'Honey there is no right way'}, {'line': "And so I fall in love just a little ol' little bit"}, {'line': 'Every day with someone new'}, {'line': "I fall in love just a little ol' little bit"}, {'line': 'Every day with someone new'}, {'line': "I fall in love just a little ol' little bit"}, {'line': 'Every day with someone new'}, {'line': "I fall in love just a little ol' little bit"}, {'line': 'Every day with someone new'}, {'line': "There's an art to life's distractions"}, {'line': 'To somehow escape the burning weight'}, {'line': 'The art of scraping through'}, {'line': 'Some like to imagine'}, {'line': 'The dark caress of someone else I guess any thrill will 

In [17]:
example.lyrics

[{'line': 'Go take this the wrong way'},
 {'line': 'You knew who I was with every step that I ran to you'},
 {'line': 'Only blue or black days'},
 {'line': 'Electing strange perfections in any stranger I choose'},
 {'line': 'Would things be easier if there was a right way'},
 {'line': 'Honey there is no right way'},
 {'line': "And so I fall in love just a little ol' little bit"},
 {'line': 'Every day with someone new'},
 {'line': "I fall in love just a little ol' little bit"},
 {'line': 'Every day with someone new'},
 {'line': "I fall in love just a little ol' little bit"},
 {'line': 'Every day with someone new'},
 {'line': "I fall in love just a little ol' little bit"},
 {'line': 'Every day with someone new'},
 {'line': "There's an art to life's distractions"},
 {'line': 'To somehow escape the burning weight'},
 {'line': 'The art of scraping through'},
 {'line': 'Some like to imagine'},
 {'line': 'The dark caress of someone else I guess any thrill will do'},
 {'line': '******* Th'}]